In [ ]:
import os
import numpy as np
import glob
import random

from keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Merge, BatchNormalization
from keras.layers.merge import Concatenate
from keras.callbacks import Callback, TensorBoard
from keras.models import Model
#from preprocessing import load_data, preprocess_data

from preprocessing import *
from crowdnet import *
from visualize import *

In [ ]:
model_name = 'dcc_crowdnet'
model_path = os.path.expanduser(os.path.join('models', model_name))
data_path = os.path.expanduser(os.path.join('~/data', model_name))
weights_path = os.path.expanduser(os.path.join('~/models', model_name))

dataset_paths = ['dataset/UCF_CC_50']

slice_w = 256
slice_h = 256

patch_w = 225
patch_h = 225

net_density_h = 28
net_density_w = 28

In [ ]:
#mean
VGG_ILSVRC_16_layers_mean = np.zeros((3, patch_h, patch_w), dtype='f4')
VGG_ILSVRC_16_layers_mean[0,:,:] = 103.939
VGG_ILSVRC_16_layers_mean[1,:,:] = 116.779
VGG_ILSVRC_16_layers_mean[2,:,:] = 123.68

In [ ]:
# Positive image and ground truth loading
X_fs = []
Y_fs = []

for path in dataset_paths:
    images, gts, densities = load_images_and_gts(path)
    X_fs += images
    Y_fs += densities

In [ ]:
# Split test an train set
from sklearn.model_selection import train_test_split

# Train 80%, Test 20%
X_fs_train, X_fs_test, Y_fs_train, Y_fs_test = train_test_split(X_fs, Y_fs, test_size=0.2)

In [ ]:
# FS: FULL SIZE
X_train, Y_train = X_fs_train, Y_fs_train
X_test, Y_test = X_fs_test, Y_fs_test

# DATA AUGMENTATION
print ('\nMultiscale pyramidal')
print ('TRAIN:')
X_train, Y_train = multiscale_pyramidal(X_train, Y_train)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = multiscale_pyramidal(X_test, Y_test)
print (len(X_test), len(Y_test))

# PATCH SIZE
print ('\nGenerate slices')
print ('TRAIN:')
X_train, Y_train = generate_slices(X_train, Y_train, slice_w=patch_w, slice_h=patch_h, offset=8)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = generate_slices(X_test, Y_test, slice_w=patch_w, slice_h=patch_h)
print (len(X_test), len(Y_test))

print ('\nFlip')
print ('TRAIN:')
X_train, Y_train = flip_slices(X_train, Y_train)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = flip_slices(X_test, Y_test)
print (len(X_test), len(Y_test))

print ('\nSamples gt distribution correction')
print ('TRAIN:')
X_train, Y_train = samples_distribution(X_train, Y_train)
print (len(X_train), len(Y_train))

print ('\nShuffle')
print ('TRAIN:')
X_train, Y_train = shuffle_slices(X_train, Y_train)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, y_test = shuffle_slices(X_test, Y_test)
print (len(X_test), len(Y_test))

In [ ]:
#Plot TRAIN samples distribution
from matplotlib import pyplot as plt

Y_train_sum = list(filter(lambda x: True, map(np.sum, Y_train)))
Y_train_sum.sort()
plt.plot(Y_train_sum)
plt.show()

In [ ]:
# Train
print ('TRAIN:')
process_dump_to_hdf5data(X_train, Y_train, data_path, 'train', VGG_ILSVRC_16_layers_mean)
# Test
print ('TEST:')
process_dump_to_hdf5data(X_test, Y_test, data_path, 'test', VGG_ILSVRC_16_layers_mean)

In [ ]:
vis_square(list_to_np_array(X_train[:20]))

In [ ]:
#visualize one random density
i = int(random.random()*len(Y_train))
print (i)
print (np.sum(Y_train[i]))
plt.imshow(np.fliplr(Y_train[i]))

In [ ]:
batch_size=1
model = create_model(batch_size)

In [ ]:
tensorboard_callback = TensorBoard(log_dir='./output', write_graph=True, write_images=True)

In [ ]:
model.fit(np.array(X_train), np.array(np.expand_dims(Y_train, axis=-1)), epochs=100000, batch_size=batch_size)